
### 1. Download the data from <a href='https://drive.google.com/file/d/15dCNcmKskcFVjs7R0ElQkR61Ex53uJpM/view?usp=sharing'>here</a>. You have to use data.csv file for this assignment
### 2. Code the model to classify data like below image. You can use any number of units in your Dense layers.

<img src='https://i.imgur.com/33ptOFy.png'>



# <font color='red'> <b>3. Writing Callbacks </b> </font>
## You have to implement the following callbacks
-  Write your own callback function, that has to print the micro F1 score and AUC score after each epoch.Do not use tf.keras.metrics for calculating AUC and F1 score.

- Save your model at every epoch if your validation accuracy is improved from previous epoch. 

- You have to decay learning based on below conditions 
        Cond1. If your validation accuracy at that epoch is less than previous epoch accuracy, you have to decrese the
               learning rate by 10%. 
        Cond2. For every 3rd epoch, decay your learning rate by 5%.
        
- If you are getting any NaN values(either weigths or loss) while training, you have to terminate your training. 

- You have to stop the training if your validation accuracy is not increased in last 2 epochs.

- Use tensorboard for every model and analyse your scalar plots and histograms. (you need to upload the screenshots and write the observations for each model for evaluation)



<pre>
<b>Model-1</b>
<pre>
1. Use tanh as an activation for every layer except output layer.
2. use SGD with momentum as optimizer.
3. use RandomUniform(0,1) as initilizer.
3. Analyze your output and training process. 
</pre>
</pre>

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.layers import Dense,Input,Activation
from tensorflow.keras.models import Model
import random as rn

In [3]:
from sklearn.model_selection import train_test_split
x = pd.read_csv('data.csv')
x.shape

print(x)
y = x['label'].values
X = x.drop(['label'], axis=1)
#train_ds, test_ds = tfds.load('x', split=['train[70%]', 'test[30%]'])
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

             f1        f2  label
0      0.450564  1.074305    0.0
1      0.085632  0.967682    0.0
2      0.117326  0.971521    1.0
3      0.982179 -0.380408    0.0
4     -0.720352  0.955850    0.0
...         ...       ...    ...
19995 -0.491252 -0.561558    0.0
19996 -0.813124  0.049423    1.0
19997 -0.010594  0.138790    1.0
19998  0.671827  0.804306    0.0
19999 -0.854865 -0.588826    0.0

[20000 rows x 3 columns]


In [4]:
from sklearn.metrics import recall_score
class LossHistory(tf.keras.callbacks.Callback):
    def __init__(self,validation_data):
      self.x_test = validation_data[0]
      self.y_test= validation_data[1]
    def on_train_begin(self, logs={}):
        ## on begin of training, we are creating a instance varible called history
        ## it is a dict with keys [loss, acc, val_loss, val_acc]
        self.history={'loss': [],'accuracy': [],'val_loss': [],'val_accuracy': [],'val_recall': []}
        
    def on_epoch_end(self, epoch, logs={}):
        true_positives=0
        ## on end of each epoch, we will get logs and update the self.history dict
        self.history['loss'].append(logs.get('loss'))
        self.history['accuracy'].append(logs.get('accuracy'))
      
        if logs.get('val_loss', -1) != -1:
            self.history['val_loss'].append(logs.get('val_loss'))
        if logs.get('val_accuracy', -1) != -1:
            self.history['val_accuracy'].append(logs.get('val_accuracy'))
        # we can get a list of all predicted values at the end of the epoch
        # we can use these predicted value and the true values to calculate any custom evaluation score if it is needed for our model
        # Here we are taking log of all true positives and then taking average of it
        y_pred= self.model.predict(self.x_test)
        y_label_pred=np.argmax(y_pred,axis=1)
        custom_score = np.log(np.sum(y_test== y_label_pred))/len(y_test)
        
        #we can also calcualte predefined metrics such as precison, recall, etc. using callbacks 
        recall = recall_score(y_test,y_label_pred,average='micro')
        self.history['val_recall'].append(recall)
        print('custom_Score: ',np.round(custom_score,5),'Recall: ',recall)
                      

In [5]:
def create_f1():
    def f1_function(y_true, y_pred):
        y_pred_binary = tf.where(y_pred>=0.5, 1., 0.)
        tp = tf.reduce_sum(y_true * y_pred_binary)
        predicted_positives = tf.reduce_sum(y_pred_binary)
        possible_positives = tf.reduce_sum(y_true)
        return tp, predicted_positives, possible_positives
    return f1_function


class F1_score(tf.keras.metrics.Metric):
    def __init__(self):
        self.validation_data=(x_test,y_test)
    def __init__(self, **kwargs):
        super().__init__(**kwargs) # handles base args (e.g., dtype)
        self.f1_function = create_f1()
        self.tp_count = self.add_weight("tp_count", initializer="zeros")
        self.all_predicted_positives = self.add_weight('all_predicted_positives', initializer='zeros')
        self.all_possible_positives = self.add_weight('all_possible_positives', initializer='zeros')

    def update_state(self, y_true, y_pred,sample_weight=None):
        tp, predicted_positives, possible_positives = self.f1_function(y_true, y_pred)
        self.tp_count.assign_add(tp)
        self.all_predicted_positives.assign_add(predicted_positives)
        self.all_possible_positives.assign_add(possible_positives)

    def result(self):
        precision = self.tp_count / self.all_predicted_positives
        recall = self.tp_count / self.all_possible_positives
        f1 = 2*(precision*recall)/(precision+recall)
        return f1
    
history_own=F1_score()

In [6]:
class Metrics(tf.keras.callbacks.Callback):
    def __init__(self):
        self.validation_data=(X_test,y_test)
    def on_train_begin(self, logs={}):
        self.val_f1s = []   
    def on_epoch_end(self, epoch, logs={}):
        val_predict = (np.asarray(self.model.predict(self.validation_data[0]))).round()
        val_targ = self.validation_data[1]
        val_f1 = f1_score(val_targ, val_predict.round())
        roc_val=roc_auc_score(val_targ, val_predict)
        self.val_f1s.append(val_f1)  
        print("-f1 score :",val_f1,"-ROCValue :", roc_val)
history_own = Metrics()

In [8]:
#Input layer
input_layer = Input(shape=(2,))
#Dense hidden layer
layer1 = Dense(50,activation='tanh',kernel_initializer=tf.keras.initializers.RandomUniform(0,1))(input_layer)
#output layer
layer2 = Dense(10,activation='tanh',kernel_initializer=tf.keras.initializers.RandomUniform(0,1))(layer1)
#Creating a model
layer3 = Dense(10,activation='tanh',kernel_initializer=tf.keras.initializers.RandomUniform(0,1))(layer2)
layer4 = Dense(10,activation='tanh',kernel_initializer=tf.keras.initializers.RandomUniform(0,1))(layer3)
layer5 = Dense(10,activation='tanh',kernel_initializer=tf.keras.initializers.RandomUniform(0,1))(layer4)
output = Dense(10,activation='tanh',kernel_initializer=tf.keras.initializers.RandomUniform(0,1))(layer5)
model = Model(inputs=input_layer,outputs=output)


#Callbacks
#history_own=LossHistory(validation_data=[X_test,y_test])            

optimizer = tf.keras.optimizers.SGD(0.01)

model.compile(optimizer=optimizer, loss='binary_crossentropy',metrics=['accuracy'])

model.fit(X_train,y_train,epochs=15, validation_data=(X_test,y_test),callbacks=[history_own])

ValueError: Error when checking input: expected input_2 to have shape (2,) but got array with shape (3,)

<pre>
<b>Model-2</b>
<pre>
1. Use relu as an activation for every layer except output layer.
2. use SGD with momentum as optimizer.
3. use RandomUniform(0,1) as initilizer.
3. Analyze your output and training process. 
</pre>
</pre>

<pre>
<b>Model-3</b>
<pre>
1. Use relu as an activation for every layer except output layer.
2. use SGD with momentum as optimizer.
3. use he_uniform() as initilizer.
3. Analyze your output and training process. 
</pre>
</pre>




<pre>
<b>Model-4</b>
<pre>
1. Try with any values to get better accuracy/f1 score.  
</pre>
</pre>

# Note 
Make sure that you are plotting tensorboard plots either in your notebook or you can try to create a pdf file with all the tensorboard screenshots.Please write your analysis of tensorboard results for each model.
